In [1]:
from bs4 import BeautifulSoup
from pandas import DataFrame
import requests
import json
import time
import csv

domain_url = "https://domainurl.com"  # replace with domain url
json_link = "/latest.json?ascending=false&no_definitions=true&page={}"

starting_url="https://domainurl.com/t/" # replace with domain url

# Crawl post URLs from the forum Json pages
def url_crawler(num):  
    ending_urls={}  # {post titile: ending url}
    is_last_page=False
    api_url = (domain_url + json_link).format(num)
    
    page = requests.get(api_url)
    page_json= page.json()  
    topic_list= page_json["topic_list"]
    topics=topic_list["topics"]

    if len(topics)>0:
        for topic in topics:     #get all post urls in this json page
            post_id= topic["id"]
            post_slug= topic["slug"] 
            post_title= topic["title"] 
            ending_urls[post_title]= post_slug+"/"+ str(post_id) 
            time.sleep(3) 
            
        if "more_topics_url" not in topic_list.keys():  #on last page
            is_last_page= True
    else:  # pages after the last page
        is_last_page= True  
    return is_last_page,ending_urls


In [2]:
# optional
forum_categories={"About the forum":["Our forum is changing",
                                     "Get help using the forum"],
                 "About me":["Introductions and personal stories"],
                 "About Parkinson's and health":["Wellbeing",
                                                 "Treatments and therapies",
                                                 "Impulsive and compulsive behaviour",
                                                 "Newly diagnosed",
                                                 "Symptoms"],
                  "Living with Parkinson's":["Carers, friends and family",
                                             "Daily life",
                                             "Work, benefits and money",
                                             "Young Onset Parkinson's"],
                  "Taking Control":["Ways to take control"],
                  "Research":["Research discussion",
                              "Research opportunities",
                              "Research Support Network"],
                  "Social and creative":[],
                  "Get involved":["News, events and taking part",
                                  "About the community"]                 
                 }

forum_subcategories=[]
for line in list(forum_categories.values()):
    for sub in line:
        forum_subcategories.append(sub)
                 


In [47]:
def get_url(tail):
    return starting_url + tail
    
def strip_span_tags(str,is_username):
    try:
        if (is_username):
            str=str.replace('<span itemprop="name">',"")  #<span itemprop="name">
        else: #category and subcategory
            str=str.replace('<span class="category-name" itemprop="name">',"")
        str=str.replace('</span>',"")  
    except AttributeError:
        str=str
    return str

# iteratively visit all post links and perform scraping via BeautifulSoap & store
def post_crawler(title,url):
    page = requests.get(url)  # get request from url
    soup = BeautifulSoup(page.content, "html.parser")# convert the text
    # print(soup)

    post_title= title  #1.title

    metadata= soup.findAll("span", itemprop="name")
    datum1= strip_span_tags(str(metadata[0]),False)  #2. category(or username1 in old posts)
    datum2= strip_span_tags(str(metadata[1]),False)   #3. subcategory (or username2)
    datum3= strip_span_tags(str(metadata[2]),True)  #4. username (or username3)
    
    # old posts do not have category and subcate
    post_category,post_subcate= None,None
    if datum1 in forum_categories.keys():  #strip them as categories first
        post_category = datum1
    if datum2 in forum_subcategories:
        post_subcate = datum2
        
    if not post_category is None and not post_subcate is None: #new posts
        post_username= datum3
    else: #old posts
        post_username=strip_span_tags(str(metadata[0]),True) #is_username
    

    post_time=soup.find("time")
    if post_time.has_attr('datetime'): 
        post_time= post_time['datetime']   #5. time
    
    post_content=soup.find("div",itemprop="articleBody")
    post_content=str(post_content)
    post_content= post_content.strip('<div class="post" itemprop="articleBody">')
    post_content= post_content.strip('<p>')
    post_content= post_content.strip('</p>')  
    post_content= post_content.strip('\n        ')
    post_content= post_content.strip('\n          ') #6. initiating post content


    post_data= {"url":url, "title":post_title, "category":post_category, 
                "subcategory":post_subcate, "username":post_username, 
                "time":post_time,"content":post_content}
    
    return post_data
    
    
def main_crawler(tails):
    batch_post_data= []  #for every 30 posts
    for title,url in tails.items():
        post_url=get_url(url)
        post_data=post_crawler(title,post_url) 
        batch_post_data.append(post_data)
    return batch_post_data
    

In [48]:
path_to_file="filepath.json"  #replace with true path
json_f= open(path_to_file, 'w') 

json_link_counter=0  
is_end=False
POST_COUNT_PER_PAGE=30

dataset={}  #{post_num:{}}

while not is_end:
    print("page: ",json_link_counter)
    is_end,url_tails=url_crawler(json_link_counter) 
    batch_data= main_crawler(url_tails)
    for i in range(len(batch_data)):
        post_num = (json_link_counter)*POST_COUNT_PER_PAGE + i +1 
        dataset[post_num] = batch_data[i]

    json_link_counter+=1

json.dump(dataset, json_f)  
json_f.close()


page:  321
page:  322
